<a href="https://colab.research.google.com/github/mazenhassan9/Cross-validation/blob/main/XGBoost_vs_LightGBM_with_K_Fold.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mounting Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Projects Data/Light GBM vs XGBoost

/content/drive/MyDrive/Projects Data/Light GBM vs XGBoost


In [ ]:
ls

survey-results-public.csv


# **Importing Important Libraries**

In [ ]:
# Import required libraries and read test and train data set. Append both.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
%matplotlib inline

# **Read Data**

In [ ]:
# read data frame using pd.read_csv
df = pd.read_csv("./survey-results-public.csv")
# print data columns names
print(df.columns.tolist())
# print the data shape
print(df.shape)
# display the first 5 rows from the data frame 
df.head(5)

['Respondent', 'Professional', 'ProgramHobby', 'Country', 'University', 'EmploymentStatus', 'FormalEducation', 'MajorUndergrad', 'HomeRemote', 'CompanySize', 'CompanyType', 'YearsProgram', 'YearsCodedJob', 'YearsCodedJobPast', 'DeveloperType', 'WebDeveloperType', 'MobileDeveloperType', 'NonDeveloperType', 'CareerSatisfaction', 'JobSatisfaction', 'ExCoderReturn', 'ExCoderNotForMe', 'ExCoderBalance', 'ExCoder10Years', 'ExCoderBelonged', 'ExCoderSkills', 'ExCoderWillNotCode', 'ExCoderActive', 'PronounceGIF', 'ProblemSolving', 'BuildingThings', 'LearningNewTech', 'BoringDetails', 'JobSecurity', 'DiversityImportant', 'AnnoyingUI', 'FriendsDevelopers', 'RightWrongWay', 'UnderstandComputers', 'SeriousWork', 'InvestTimeTools', 'WorkPayCare', 'KinshipDevelopers', 'ChallengeMyself', 'CompetePeers', 'ChangeWorld', 'JobSeekingStatus', 'HoursPerWeek', 'LastNewJob', 'AssessJobIndustry', 'AssessJobRole', 'AssessJobExp', 'AssessJobDept', 'AssessJobTech', 'AssessJobProjects', 'AssessJobCompensation', '

,Respondent,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,CompanyType,YearsProgram,YearsCodedJob,YearsCodedJobPast,DeveloperType,WebDeveloperType,MobileDeveloperType,NonDeveloperType,CareerSatisfaction,JobSatisfaction,ExCoderReturn,ExCoderNotForMe,ExCoderBalance,ExCoder10Years,ExCoderBelonged,ExCoderSkills,ExCoderWillNotCode,ExCoderActive,PronounceGIF,ProblemSolving,BuildingThings,LearningNewTech,BoringDetails,JobSecurity,DiversityImportant,AnnoyingUI,FriendsDevelopers,RightWrongWay,UnderstandComputers,SeriousWork,...,InfluenceInternet,InfluenceWorkstation,InfluenceHardware,InfluenceServers,InfluenceTechStack,InfluenceDeptTech,InfluenceVizTools,InfluenceDatabase,InfluenceCloud,InfluenceConsultants,InfluenceRecruitment,InfluenceCommunication,StackOverflowDescribes,StackOverflowSatisfaction,StackOverflowDevices,StackOverflowFoundAnswer,StackOverflowCopiedCode,StackOverflowJobListing,StackOverflowCompanyPage,StackOverflowJobSearch,StackOverflowNewQuestion,StackOverflowAnswer,StackOverflowMetaChat,StackOverflowAdsRelevant,StackOverflowAdsDistracting,StackOverflowModeration,StackOverflowCommunity,StackOverflowHelpful,StackOverflowBetter,StackOverflowWhatDo,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers,Salary,ExpectedSalary
0,1,Student,"Yes, both",United States,No,"Not employed, and not looking for work",Secondary school,NaN,NaN,NaN,NaN,2 to 3 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"With a soft ""g,"" like ""jiff""",Strongly agree,Strongly agree,Agree,Disagree,Strongly agree,Agree,Agree,Disagree,Somewhat agree,Disagree,Strongly agree,...,Not very satisfied,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have created a CV or Developer Story on Stac...,9.0,Desktop; iOS app,At least once each week,Haven't done at all,Once or twice,Haven't done at all,Haven't done at all,Several times,Several times,Once or twice,Somewhat agree,Strongly disagree,Strongly disagree,Strongly agree,Agree,Strongly agree,Strongly agree,Strongly disagree,Male,High school,White or of European descent,Strongly disagree,Strongly agree,Disagree,Strongly agree,NaN,NaN
1,2,Student,"Yes, both",United Kingdom,"Yes, full-time",Employed part-time,Some college/university study without earning ...,Computer science or software engineering,"More than half, but not all, the time",20 to 99 employees,"Privately-held limited company, not in startup...",9 to 10 years,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"With a hard ""g,"" like ""gift""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Satisfied,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,No influence at all,I have created a CV or Developer Story on Stac...,8.0,Desktop; iOS browser; iOS app; Android browser...,Several times,Several times,Once or twice,Once or twice,Once or twice,Haven't done at all,Several times,At least once each week,Disagree,Strongly disagree,Strongly disagree,Strongly agree,Agree,Strongly agree,Strongly agree,Strongly disagree,Male,A master's degree,White or of European descent,Somewhat agree,Somewhat agree,Disagree,Strongly agree,NaN,37500.0
2,3,Professional developer,"Yes, both",United Kingdom,No,Employed full-time,Bachelor's degree,Computer science or software engineering,"Less than half the time, but at least one day ...","10,000 or more employees",Publicly-traded corporation,20 or more years,20 or more years,NaN,Other,NaN,NaN,NaN,8.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"With a hard ""g,"" like ""gift""",Strongly agree,Strongly agree,Strongly agree,Somewhat agree,Agree,Strongly agree,Agree,Somewhat agree,Disagree,Disagree,Agree,...,Very satisfied,A lot of influence,Some influence,Some influence,Some influence,A lot of influence,Some influence,Some i

# **Clean Data**

In [ ]:
def clean_data(df):
    '''
    INPUT
    df - pandas dataframe 
    
    OUTPUT
    X - A matrix holding all of the variables you want to consider when predicting the response
    y - the corresponding response vector
    '''
    # Drop rows with missing salary values
    df = df.dropna(subset=['Salary'], axis=0)
    y = df['Salary']
    
    #Drop respondent and expected salary columns
    df = df.drop(['Respondent', 'ExpectedSalary', 'Salary'], axis=1)
    
    # Fill numeric columns with the mean
    num_vars = df.select_dtypes(include=['float', 'int']).columns
    for col in num_vars:
        df[col].fillna((df[col].mean()), inplace=True)
        
    # Dummy the categorical variables
    cat_vars = df.select_dtypes(include=['object']).copy().columns
    for var in  cat_vars:
        # replace Nan value with nan string 
        df[var].fillna("nan",inplace=True)
        # for each cat apply label encoding 
        lb=LabelEncoder()
        df[var]=lb.fit_transform(df[var])
        
    
    X = df
    return X, y

In [ ]:
#Use the function to create X and y
X, y = clean_data(df)
X.head()    

,Professional,ProgramHobby,Country,University,EmploymentStatus,FormalEducation,MajorUndergrad,HomeRemote,CompanySize,CompanyType,YearsProgram,YearsCodedJob,YearsCodedJobPast,DeveloperType,WebDeveloperType,MobileDeveloperType,NonDeveloperType,CareerSatisfaction,JobSatisfaction,ExCoderReturn,ExCoderNotForMe,ExCoderBalance,ExCoder10Years,ExCoderBelonged,ExCoderSkills,ExCoderWillNotCode,ExCoderActive,PronounceGIF,ProblemSolving,BuildingThings,LearningNewTech,BoringDetails,JobSecurity,DiversityImportant,AnnoyingUI,FriendsDevelopers,RightWrongWay,UnderstandComputers,SeriousWork,InvestTimeTools,...,EquipmentSatisfiedStorage,EquipmentSatisfiedRW,InfluenceInternet,InfluenceWorkstation,InfluenceHardware,InfluenceServers,InfluenceTechStack,InfluenceDeptTech,InfluenceVizTools,InfluenceDatabase,InfluenceCloud,InfluenceConsultants,InfluenceRecruitment,InfluenceCommunication,StackOverflowDescribes,StackOverflowSatisfaction,StackOverflowDevices,StackOverflowFoundAnswer,StackOverflowCopiedCode,StackOverflowJobListing,StackOverflowCompanyPage,StackOverflowJobSearch,StackOverflowNewQuestion,StackOverflowAnswer,StackOverflowMetaChat,StackOverflowAdsRelevant,StackOverflowAdsDistracting,StackOverflowModeration,StackOverflowCommunity,StackOverflowHelpful,StackOverflowBetter,StackOverflowWhatDo,StackOverflowMakeMoney,Gender,HighestEducationParents,Race,SurveyLong,QuestionsInteresting,QuestionsConfusing,InterestedAnswers
2,0,3,126,1,0,0,8,4,2,6,12,12,4,333,3,8,0,8.0,9.0,1,0,0,1,0,1,0,0,2,3,3,3,2,0,3,0,2,1,1,0,2,...,2,3,4,0,4,4,4,0,4,4,4,4,4,4,1,8.000000,21,3,2,2,2,2,2,0,0,1,1,4,3,0,0,0,1,6,3,51,2,0,1,0
14,0,2,126,1,0,6,6,2,5,6,12,12,4,96,3,8,0,8.0,8.0,1,0,0,1,0,1,0,0,2,3,3,0,1,2,2,3,2,2,1,0,0,...,3,2,3,0,3,3,2,3,2,2,2,3,2,2,1,8.000000,21,4,3,4,4,4,3,0,4,1,1,1,0,0,3,0,1,6,4,51,2,0,1,0
17,0,3,127,3,0,0,8,2,0,6,12,12,4,507,3,8,0,9.0,8.0,1,0,0,1,0,1,0,0,2,3,3,3,4,0,0,0,1,1,3,0,0,...,3,1,3,0,4,3,4,0,3,4,0,3,3,0,1,8.000000,5,1,1,3,2,2,1,1,2,2,1,1,0,3,3,3,1,6,2,48,1,0,1,0
18,0,2,127,1,0,0,8,0,2,6,11,13,4,343,2,8,0,5.0,3.0,1,0,0,1,0,1,0,0,2,3,3,3,1,3,0,2,0,1,1,0,0,...,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,8.511858,32,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,5,19,10,55,5,5,5,5
22,0,0,55,1,0,0,6,0,6,5,1,15,4,333,3,8,0,8.0,9.0,1,0,0,1,0,1,0,0,2,5,5,5,5,5,5,5,5,5,5,5,5,...,4,4,4,0,4,2,2,0,2,2,2,2,2,2,0,8.000000,14,0,3,2,2,2,3,2,2,2,1,1,0,3,3,0,2,6,0,51,3,2,2,0


# **Train/Test Split**

In [ ]:
# split the data to train and test using train_test_split with test_size =0.2 and seed=42 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =.2,random_state=42)
# print X_train shape and Y_test shape 
print(X_train.shape,X_test.shape )  

(10312, 151) (2579, 151)


# **Model with train validation split **

In [ ]:
# split the train data  to  training data and validation data  validation size =0.2 and seed=42 
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size =.2,random_state=42)
# print X_train shape and Y_test shape 
print(X_train.shape,X_val.shape )  

(8249, 151) (2063, 151)


### XGBOOST


#### train

In [ ]:
# import xgboost as xgb 
import xgboost as xgb


In [ ]:

# convert  train validation and test data  to xgboost data structure using DMatrix please refer to the documentation 

dtrain = xgb.DMatrix(data=X_train,label=y_train)
dval =  xgb.DMatrix(data=X_val,label=y_val)
dtest =  xgb.DMatrix(data=X_test,label=y_test)

# specify parameters via map set the objective to regression loss function  please refer to the documentation  
param = {'max_depth':6, 'eta':0.01,'seed':0,
         'objective':'reg:squarederror',
         'subsample':0.9,
         'colsample_bytree':0.98,
         "metrics":'rmse'
         }
# set nbr of iteration to 1000000000 
num_iter = 1000000000
# fit  the model and check the requirements params and set early stopping round params to 20 rounds 
xgb_model=xgb.train(params=param,
                    dtrain=dtrain,
                    num_boost_round= num_iter,
                    evals=[(dtrain,"train"),(dval,"validation")],
                    early_stopping_rounds=100,
                    verbose_eval=100)


[0]	train-rmse:68154.3	validation-rmse:68530.3
Multiple eval metrics have been passed: 'validation-rmse' will be used for early stopping.

Will train until validation-rmse hasn't improved in 100 rounds.
[100]	train-rmse:32932.2	validation-rmse:33646.5
[200]	train-rmse:22670.1	validation-rmse:24328
[300]	train-rmse:19326.1	validation-rmse:21939.2
[400]	train-rmse:17691	validation-rmse:21167
[500]	train-rmse:16556.6	validation-rmse:20852.6
[600]	train-rmse:15618.4	validation-rmse:20644
[700]	train-rmse:14805.3	validation-rmse:20493.2
[800]	train-rmse:14039.1	validation-rmse:20365.5
[900]	train-rmse:13368.3	validation-rmse:20270.8
[1000]	train-rmse:12732.3	validation-rmse:20189.3
[1100]	train-rmse:12143.6	validation-rmse:20123.8
[1200]	train-rmse:11614.3	validation-rmse:20060.3
[1300]	train-rmse:11106.1	validation-rmse:20014.1
[1400]	train-rmse:10622.6	validation-rmse:19967.8
[1500]	train-rmse:10158	validation-rmse:19945.6
[1600]	train-rmse:9735.47	validation-rmse:19920.6
[1700]	train-rms

####  inference

In [ ]:
# Let's predict the train validation test 
xgb_train_pred = xgb_model.predict(dtrain)
xgb_validation_pred = xgb_model.predict(dval)
xgb_test_pred =  xgb_model.predict(dtest)

### LGBM

#### train

In [ ]:
# import lightgbm as lgbm 
import lightgbm as lgbm

In [ ]:
# convert  train data and test data  to xgboost data structure using DMatrix please refer to the documentation 
dtrain = lgbm.Dataset(X_train,y_train)
dval = lgbm.Dataset(X_val,y_val)
dtest =  lgbm.Dataset(X_test,y_test)
# specify parameters via map set the objective to regression loss function  please refer to the documentation  
params = {
    'objective': 'regression',
    'metric': "rmse",
    'num_leaves': 31,
    'learning_rate': 0.01,
    "max_depth":6,
    "seed":0
}
# set nbr of iteration to 400 
num_iter = 10000000
# fit  the model and check the requirements params and set early stopping  rounds 
gbm = lgbm.train(params=params,
                train_set=dtrain,
                num_boost_round=num_iter,
                valid_sets=[dtrain,dval],
                 verbose_eval=100,
                 early_stopping_rounds=50)

Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 26513.4	valid_1's rmse: 26597.9
[200]	training's rmse: 22379.6	valid_1's rmse: 23064.9
[300]	training's rmse: 20385.3	valid_1's rmse: 21655.9
[400]	training's rmse: 19126.6	valid_1's rmse: 21034.2
[500]	training's rmse: 18223.6	valid_1's rmse: 20726
[600]	training's rmse: 17477.9	valid_1's rmse: 20539.9
[700]	training's rmse: 16812.3	valid_1's rmse: 20383.9
[800]	training's rmse: 16224.3	valid_1's rmse: 20251.1
[900]	training's rmse: 15683.6	valid_1's rmse: 20157.2
[1000]	training's rmse: 15182.8	valid_1's rmse: 20083.8
[1100]	training's rmse: 14712.9	valid_1's rmse: 20000.9
[1200]	training's rmse: 14275.9	valid_1's rmse: 19944
[1300]	training's rmse: 13872.5	valid_1's rmse: 19906.8
[1400]	training's rmse: 13496.4	valid_1's rmse: 19871.1
[1500]	training's rmse: 13143.2	valid_1's rmse: 19851.7
[1600]	training's rmse: 12800.9	valid_1's rmse: 19830.7
[1700]	training's rmse: 12481.1	valid_1's rmse: 19809.6

####  inference

In [ ]:
# Let's predict the train validation test 
lgbm_train_pred = gbm.predict(X_train)
lgbm_val_pred = gbm.predict(X_val)
lgbm_test_pred =  gbm.predict(X_test)

### Evalution 

In [ ]:
def eval_results(train_pred,train_label,
                 val_pred,val_label,
                 test_pred,test_label,
                 model_name) : 
  eval_dict={}
  eval_dict["model_name"]=model_name
  eval_dict["train_mean_squared_error"]=mean_squared_error(train_pred,train_label)
  eval_dict["val_mean_squared_error"]=mean_squared_error(val_pred,val_label)
  eval_dict["test_mean_squared_error"]=mean_squared_error(test_pred,test_label)

  eval_dict["train_mean_absolute_error"]=mean_absolute_error(train_pred,train_label)
  eval_dict["val_mean_absolute_error"]=mean_absolute_error(val_pred,val_label)
  eval_dict["test_mean_absolute_error"]=mean_absolute_error(test_pred,test_label)

  eval_dict["train_r2_score"]=r2_score(train_pred,train_label)
  eval_dict["val_r2_score"]=r2_score(val_pred,val_label)
  eval_dict["test_r2_score"]=r2_score(test_pred,test_label)
  return eval_dict

In [ ]:
# create  a list of dicts contains the evalation of each model using eval_results funcation 
list_results=[eval_results(xgb_train_pred,y_train,
                           xgb_validation_pred,y_val, 
                           xgb_test_pred,y_test,
                           "xgb"),
              eval_results(lgbm_train_pred,y_train,
                           lgbm_val_pred,y_val,
                           lgbm_test_pred,y_test,
                           "lgbm"),
          

 ]
# convert the list of dicts to data frame 
result_df=pd.DataFrame(list_results)
result_df.sort_values("test_mean_absolute_error")

,model_name,train_mean_squared_error,val_mean_squared_error,test_mean_squared_error,train_mean_absolute_error,val_mean_absolute_error,test_mean_absolute_error,train_r2_score,val_r2_score,test_r2_score
0,xgb,3.810482e+07,3.933964e+08,3.805373e+08,4404.064341,13889.271018,14048.963151,0.972984,0.648508,0.652690
1,lgbm,1.449097e+08,3.918374e+08,3.869081e+08,8522.363805,13971.653831,14225.910995,0.884641,0.650360,0.648426


# **Model With K-Fold**

In [ ]:
# split the data to train and test using train_test_split with test_size =0.2 and seed=42 
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =.2,random_state=42)
# print X_train shape and Y_test shape 
print(X_train.shape,X_test.shape )  

(10312, 151) (2579, 151)


In [ ]:
# create Kfold instance  with seed = 1994 and K=5 ,shuffle=True
kf=KFold(n_splits=5,random_state=1994,shuffle=True)

In [ ]:
# specify parameters via map set the objective to regression loss function  please refer to the documentation  
param = {'max_depth':6, 'eta':0.01,'seed':0,
         'objective':'reg:squarederror',
         'subsample':0.9,
         'colsample_bytree':0.98,
         "metrics":'rmse'
         }

In [ ]:
# create  func which take as input X_train,y_train,X_val,y_val,X_test to train xgb model and return  a rmse score and the test predection 
def train_xgboost(X_train,y_train,X_val,y_val,X_test) : 
  # create DMatrix 
  dtrain = xgb.DMatrix(data=X_train,label=y_train)
  dval =  xgb.DMatrix(data=X_val,label=y_val)
  dtest =  xgb.DMatrix(data=X_test)
  # fit the model 
  xgb_model=xgb.train(params=param,
                    dtrain=dtrain,
                    num_boost_round= num_iter,
                    evals=[(dtrain,"train"),(dval,"validation")],
                    early_stopping_rounds=100,
                    verbose_eval=200)
  # predcit validation label 
  val_pred=xgb_model.predict(dval)
  #get validation rmse 
  validation_score=np.sqrt(mean_squared_error(y_val,val_pred))
  # predcit the test data 
  test_pred=xgb_model.predict(dtest)
  return validation_score,test_pred

In [ ]:
# set nbr of iteration to 1000000000 
num_iter = 100000000
# list to hold the validation score for each fold
val_scores=[]
# a 0 array to hold the test predection 
test_preds=np.zeros_like(y_test)
for i,(train_index,validation_index) in enumerate(kf.split(X_train)) : 
  # slice  train data and validation data for each fold using the index 
  fold_X_train=X_train.iloc[train_index]
  fold_X_vali=X_train.iloc[validation_index]
  fold_Y_train=y_train.iloc[train_index]
  fold_Y_vali=y_train.iloc[validation_index]
  print("#"*10+"Fold {}".format(str(i+1))+"#"*10)
  # train the model 
  validation_score,test_pred=train_xgboost(fold_X_train,fold_Y_train,fold_X_vali,fold_Y_vali,X_test)
  # append the fold validation score 
  val_scores.append(validation_score)
  # append the test prediction 
  test_preds+=test_preds/5
  print(" rmse score = {}".format(validation_score))
print(" Cross validation score = {}".format(np.mean(val_scores)))

##########Fold 1##########
[0]	train-rmse:68061.2	validation-rmse:68894.9
Multiple eval metrics have been passed: 'validation-rmse' will be used for early stopping.

Will train until validation-rmse hasn't improved in 100 rounds.
[200]	train-rmse:22718.4	validation-rmse:24330.7
[400]	train-rmse:17735.5	validation-rmse:20941.9
[600]	train-rmse:15620.6	validation-rmse:20309.8
[800]	train-rmse:14086.9	validation-rmse:19995.9
[1000]	train-rmse:12789.9	validation-rmse:19803.3
[1200]	train-rmse:11666.7	validation-rmse:19673.9
[1400]	train-rmse:10700.9	validation-rmse:19578.2
[1600]	train-rmse:9836.32	validation-rmse:19510.3
[1800]	train-rmse:9089.67	validation-rmse:19482.6
[2000]	train-rmse:8358.26	validation-rmse:19463.9
[2200]	train-rmse:7728.94	validation-rmse:19447.1
[2400]	train-rmse:7165.53	validation-rmse:19425.8
[2600]	train-rmse:6636.66	validation-rmse:19416
[2800]	train-rmse:6160.67	validation-rmse:19402.4
[3000]	train-rmse:5723.3	validation-rmse:19394.9
[3200]	train-rmse:5313.1	va

In [ ]:
print("mean_squared_error",mean_squared_error(test_preds,))
print("mean_absolute_error",mean_absolute_error(test_preds,y_test))
print("r2_score",r2_score(test_preds,y_test))

mean_squared_error 4851211166.7413845
mean_absolute_error 57064.22491187716
r2_score 0.0
